1. For each year and month (e.g January 2020 => “2020-01-01” or “2020-01” or “Jan 2020”:

What was the total number of trips?

Which day of week (e.g. monday, tuesday, etc..) had the most trips?

Which hour of the day had the most trips?

What was the average number of passengers?

What was the average amount paid per trip (using total_amount)?

What was the average amount paid per passenger (using total_amount)?

In [0]:
#reading file from dbfs
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession

df_path= "/dbfs/mnt/bde2/combined_df"
combined_df= spark.read.parquet(df_path)

In [0]:
#create temp table
temp_table_name = 'combined_df'
combined_df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql
-- Create or replace a temporary view named 'year_month_table'.
-- This view will store the result of a query temporarily for the duration of the session.
CREATE OR REPLACE TEMP VIEW year_month_table AS
SELECT
  tpep_pickup_datetime,
  passenger_count,
  total_amount
FROM combined_df;

In [0]:
%sql
-- Common Table Expression (CTE) named MonthlyStats:
-- This CTE extracts and aggregates data from the 'year_month_table'.
WITH MonthlyStats AS (
  SELECT 
    DATE_FORMAT(tpep_pickup_datetime, "yyyy-MM") AS year_month,
    DATE_FORMAT(tpep_pickup_datetime, "EEEE") AS day_of_week,
    EXTRACT(HOUR FROM tpep_pickup_datetime) AS hour_of_day,
    COUNT(*) AS total_trips,
    AVG(passenger_count) AS avg_passenger_count,
    AVG(total_amount) AS avg_total_amount,
    AVG(total_amount / passenger_count) AS avg_amount_per_passenger
  FROM
    year_month_table --temp table
  GROUP BY
    year_month, day_of_week, hour_of_day
),
-- CTE named TotalTrips:
-- This CTE calculates the total number of trips for each month.
TotalTrips AS (
  SELECT
    year_month,
    SUM(total_trips) AS year_month_total_trips
  FROM
    MonthlyStats
  GROUP BY
    year_month
)
-- Main query:
-- This part of the query selects specific columns from the MonthlyStats CTE
-- and joins it with the TotalTrips CTE to get the total trips for each month.
SELECT 
SELECT 
  M1.year_month,
  M1.day_of_week,
  M1.hour_of_day,
  M1.total_trips,
  M1.avg_passenger_count,
  M1.avg_total_amount,
  M1.avg_amount_per_passenger,
  T1.year_month_total_trips AS total_trips_year_month
FROM
  MonthlyStats M1
JOIN
  TotalTrips T1
ON
  M1.year_month = T1.year_month
WHERE
  (M1.year_month, M1.total_trips) IN (
    -- Subquery: It selects the maximum total trips for each month.
    SELECT 
      M2.year_month,
      MAX(M2.total_trips) AS max_trips
    FROM
      MonthlyStats M2
    WHERE
      M1.year_month = M2.year_month
    GROUP BY
      M2.year_month
  )
ORDER BY
  M1.year_month; -- Orders the results by year_month

year_month,day_of_week,hour_of_day,total_trips,avg_passenger_count,avg_total_amount,avg_amount_per_passenger,total_trips_year_month
2015-01,Friday,19,148571,1.3233740097327205,15.067663339423193,13.121540763193343,12126713
2015-02,Friday,19,123224,1.2999091086152048,16.060129844840827,14.102754592795597,11922556
2015-03,Sunday,0,142680,1.3880081300813008,15.815984861229863,13.392446590186927,12852284
2015-04,Thursday,19,141749,1.289927971273166,16.216791017929477,14.283314440862487,12604849
2015-05,Friday,19,141199,1.3297473778142905,16.142677497723877,13.997675542998412,12809867
2015-06,Tuesday,19,123466,1.281705084800674,15.863494322324714,14.023058554873375,11935923
2015-07,Wednesday,19,124139,1.291809987191777,15.84360942169782,13.944513502881327,11201027
2015-08,Saturday,23,113019,1.4041621320308975,16.00760845521356,13.47963825918213,10844831
2015-09,Wednesday,19,120901,1.286052224547357,16.30648596785987,14.401260490764297,10865303
2015-10,Friday,19,130949,1.3206897341713186,16.688364554146315,14.493702452867876,11923562
